## CS412 Machine Learning HW-2 Kanat Özgen 00030973

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, Math, Latex
display(Math(r'\text{I Love }\LaTeX'))

<IPython.core.display.Math object>

In [2]:
df_convRates = pd.read_csv("conversionRates.csv")
df_freeFormResp = pd.read_csv("freeFormResponses.csv")
df_multChoiceResp = pd.read_csv("multipleChoiceResponses.csv", encoding='latin-1')
df_schema = pd.read_csv("schema.csv")

C:\Users\koezgen\AppData\Local\Temp\ipykernel_27552\3948734810.py:2: DtypeWarning: Columns (5,17,21,38,50) have mixed types. Specify dtype option on import or set low_memory=False.
  df_freeFormResp = pd.read_csv("freeFormResponses.csv")
C:\Users\koezgen\AppData\Local\Temp\ipykernel_27552\3948734810.py:3: DtypeWarning: Columns (31,83,86,87,98,99,109,116,123,124,127,129,130,164) have mixed types. Specify dtype option on import or set low_memory=False.
  df_multChoiceResp = pd.read_csv("multipleChoiceResponses.csv", encoding='latin-1')


In [3]:
df_convRates.head(10)

,Unnamed: 0,originCountry,exchangeRate
0,1,USD,1.000000
1,2,EUR,1.195826
2,3,INR,0.015620
3,4,GBP,1.324188
4,5,BRL,0.321350
5,6,RUB,0.017402
6,7,CAD,0.823688
7,8,AUD,0.802310
8,9,JPY,0.009108
9,10,CNY,0.153000


In [4]:
pd.set_option('display.max_columns', None)

In [5]:
def convert_compensation(comp):
    try:
        return float(comp.replace(',', ''))
    except:
        return np.nan

In [6]:
def convertToUSD(df, df_convRates):
    # Work on a copy to avoid modifying the original DataFrame
    df_copy = df.copy()

    # Convert 'CompensationAmount' to float
    df_copy['CompensationAmount'] = df_copy['CompensationAmount'].apply(convert_compensation)

    # Merge the conversion rates into the copy DataFrame
    merged_df = df_copy.merge(df_convRates, left_on='CompensationCurrency', right_on='originCountry', how='left')

    # Identify rows with valid exchange rates and not in USD
    valid_exchange_condition = (merged_df['CompensationCurrency'] != 'USD') & (~merged_df['exchangeRate'].isna())

    # Apply the conversion to USD only for valid exchange rates
    merged_df.loc[valid_exchange_condition, 'CompensationAmount'] *= merged_df.loc[valid_exchange_condition, 'exchangeRate']

    # Set 'CompensationCurrency' to 'USD' for these rows
    merged_df.loc[valid_exchange_condition, 'CompensationCurrency'] = 'USD'

    # Handle rows where the exchange rate was not found
    missing_rates = merged_df[~valid_exchange_condition & (~merged_df['exchangeRate'].isna())]
    if not missing_rates.empty:
        print("Missing exchange rates for currencies:", missing_rates['CompensationCurrency'].unique())

    # Drop the extra columns from the merge
    merged_df.drop(['originCountry', 'exchangeRate'], axis=1, inplace=True)

    return merged_df

# Apply the conversion function
df_multChoiceResp = convertToUSD(df_multChoiceResp, df_convRates)

Missing exchange rates for currencies: ['USD']


In [7]:
df_multChoiceResp[['CompensationAmount', 'CompensationCurrency']].head(10)

,CompensationAmount,CompensationCurrency
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,250000.0,USD
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,64184.8,USD
9,20882.4,USD


In [8]:
df_multChoiceResp["MajorSelect"].unique()

array(['Management information systems', 'Computer Science',
       'Engineering (non-computer focused)', 'Mathematics or statistics',
       'Electrical Engineering', nan, 'Other', 'Biology', 'Physics',
       'Information technology, networking, or system administration',
       'A social science', 'A health science', 'A humanities discipline',
       'Fine arts or performing arts', 'I never declared a major',
       'Psychology'], dtype=object)

In [9]:
df_multChoiceResp["FormalEducation"].unique()

array(["Bachelor's degree", "Master's degree", 'Doctoral degree', nan,
       "Some college/university study without earning a bachelor's degree",
       'I did not complete any formal education past high school',
       'Professional degree', 'I prefer not to answer'], dtype=object)

### 2.a What is the probability that a respondent is currently employed as a Programmer given they use C/C++ at work?

In [10]:
df_cpp = df_multChoiceResp[df_multChoiceResp["WorkToolsSelect"].apply(lambda x: "C/C++" in x if not pd.isna(x) else False)]
df_cpp_programmer = df_cpp[df_cpp["CurrentJobTitleSelect"] == "Programmer"]
print(df_cpp_programmer.shape[0]/df_cpp.shape[0])

0.02225130890052356


### 2.b What is the probability that a respondent is a Data Scientist given they have majored in computer science, mathematics or statistics?

In [11]:
df_cs = df_multChoiceResp[df_multChoiceResp["MajorSelect"].isin(["Computer Science", "Mathematics or statistics"])]
df_ds = df_cs[df_cs["CurrentJobTitleSelect"] == "Data Scientist"]
print(df_ds.shape[0]/df_cs.shape[0])

0.1597400634728729


### 2.c What is the probability that a respondent works in the Technology industry given that they earn more than 40,000 USD annually?

In [12]:
df_fortyk = df_multChoiceResp[df_multChoiceResp["CompensationAmount"] > 40000.0]

df_tech = df_fortyk[df_fortyk["EmployerIndustry"] == "Technology"]
print(df_tech.shape[0] / df_fortyk.shape[0])

0.1966501240694789


### 2.d What is the joint probability of a respondent being over 30 years old and having a at least a Bachelor’s degree?

In [13]:
df_age = df_multChoiceResp[df_multChoiceResp["Age"].apply(lambda x: x > 30)]
df_bsc = df_age[df_age["FormalEducation"].isin(["Bachelor's degree", "Master's degree", "Doctoral degree", "Professional degree"])]

print(df_bsc.shape[0] / df_multChoiceResp.shape[0])

0.38687485044268966


### 2.e What is the probability that a respondent is a Data Scientist who majored in Computer Science, Mathematics or statistics?

In [14]:
df_ds = df_multChoiceResp[df_multChoiceResp["CurrentJobTitleSelect"] == "Data Scientist"]
df_cs = df_ds[df_ds["MajorSelect"].isin(["Computer Science", "Mathematics or statistics"])]

print(df_cs.shape[0] / df_multChoiceResp.shape[0])

0.06323283082077052


### 2.f What is the joint probability that a respondent is from France, earns less than 100,000 USD annually, and uses Cross-Validation Often or Most of the time?

In [15]:
df_fr = df_multChoiceResp[df_multChoiceResp["Country"] == "France"]
df_fr = df_fr[df_fr["CompensationAmount"] < 100000.0]
df_fr = df_fr[df_fr["WorkMethodsFrequencyCross-Validation"].isin(["Often", "Most of the time"])]

print(df_fr.shape[0] / df_multChoiceResp.shape[0])

0.005144771476429768


### 2.g What is the probability that a respondent uses C/C++ at work given that they are employed as a Programmer? (Hint: Use your findings from Question 2a).

In [17]:
display(Math(r'P(Y = UseCpp | X = Employed) = \frac{P(X = Employed | Y = UseCpp) \cdot P(Y = UseCpp)}{P(X = Employed)}'))

<IPython.core.display.Math object>

In [18]:
df_programmer = df_multChoiceResp[df_multChoiceResp["CurrentJobTitleSelect"] == "Programmer"]
print((df_cpp_programmer.shape[0]/df_cpp.shape[0]) * (df_cpp.shape[0]/df_multChoiceResp.shape[0]) / (df_programmer.shape[0]/df_multChoiceResp.shape[0]))

0.07359307359307359


### 2.h Given the probability of a respondent wearing glasses is 0.15, and the probability of a respondent wearing glasses given they have a PhD is 0.25, find the probability of a respondent having a PhD given that they wear glasses.

In [19]:
display(Math(r'P(Y = PhD | X = Glasses) = \frac{P(X = Glasses | Y = PhD) \cdot P(Y = PhD)}{P(X = Glasses)} \\ \\ \frac{P(X = Glasses | Y = PhD)}{P(X = Glasses)} = 0.25/0.15 = 1.67 \\ \\ P(Y = PhD | X = Glasses) = 1.67 \cdot P(Y = PhD)'))

<IPython.core.display.Math object>

In [20]:
df_phd = df_multChoiceResp[df_multChoiceResp["FormalEducation"] == "Doctoral degree"]
print(1.67 * (df_phd.shape[0]/df_multChoiceResp.shape[0]))

0.23447535295525246
